### Long text sentimental analysis

- BERT is restricted in terms of the length of text you can feed it for analysis. It does not support texts with length > 512! 
- How to analyse longer paragraphs or articles? With some articles finding the sentiment is unclear by assessing just the first 512 words. So, there should be a way to account for larger inputs
- Two ways to solve this problem. (1) Neural text summarization to shorten the text sucj that length is < 512 or (2) Windowing method
- Use windowing method. Window text into 512 chunks > calculate sentiment on each chunk > average the sentiment across the chunks > final sentiment


#### Import model and define sentiment analysis function (check project1)

In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# initialise model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

# function for processing input text
def sentiment(tokens):
    # get output logits from model
    output = model(**tokens)
    # convert to probablities
    probs = torch.nn.functional.softmax(output[0], dim=-1)
    return probs

In [3]:
txt = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.”
"""


In [4]:
tokens = tokenizer.encode_plus(txt, special_tokens=False)
len(tokens['input_ids'])

Keyword arguments {'special_tokens': False} not recognized.
Token indices sequence length is longer than the specified maximum sequence length for this model (1347 > 512). Running this sequence through the model will result in indexing errors


1347

### Window Method

BERT cannot take texts with length > 512. So we split this text into chunks of 512 tokens at a time, and calculate our sentiment probabilities for each chunk seperately.

Because we are taking this slightly different approach, we have encoded our tokens using a different set of parameters to what we have used before. This time, we:

- Avoided adding special tokens add_special_tokens=False because this will add [CLS] and [SEP] tokens to the start and end of the full tokenized tensor of length 1345, we will instead add them manually later.

- We will not specify max_length, truncation, or padding parameters (as we do not use any of them here).

- We will return standard Python lists rather than tensors by not specifying return_tensors (it will return lists by default). This will make the following logic steps easier to follow - but we will rewrite them using PyTorch code in the next section.

In [5]:
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [7]:
start = 0
window_size = 512

total_len = len(input_ids)

loop = True

while loop:
    end = start + window_size
    if end >= total_len:
        loop = False
        end = total_len
    print(f"{start=}\n{end=}")
    start = end

start=0
end=512
start=512
end=1024
start=1024
end=1347


This logic works for shifting our window across the full length of input IDs, so now we can modify it to iterately predict sentiment for each window. There will be a few added steps for us to get this to work:

- Extract the window from input_ids and attention_mask.

- Add the start of sequence token [CLS]/101 and seperator token [SEP]/102.

- Add padding (only applicable to final batch).

- Format into dictionary containing PyTorch tensors.

- Make logits predictions with the model.

- Calculate softmax and append softmax vector to a list probs_list.

In [9]:
probs_list = []

start = 0
window_size = 510

total_len = len(input_ids)

loop = True

while loop:
    end = start + window_size
    if end >= total_len:
        loop = False
        end = total_len
        
    # extract input ids and attenion mask
    input_ids_chunk = input_ids[start:end]
    attention_mask_chunk = attention_mask[start:end]
    # add [CLS] and [SEP]
    input_ids_chunk = [101] + input_ids_chunk + [102]
    attention_mask_chunk = [1] + attention_mask_chunk + [1]
    # add padding upto window_size + 2 to get 512 tokens
    input_ids_chunk += [0] * (window_size - len(input_ids_chunk) + 2)
    attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) + 2)
    # format into pytorch tensors dictionary
    input_dict = {
        'input_ids': torch.Tensor([input_ids_chunk]).long(),
        'attention_mask': torch.Tensor([attention_mask_chunk]).int()
    }
    # make logits predictions
    probs_list.append(sentiment(input_dict))
    
    print(f"{start=}\n{end=}")
    start = end

start=0
end=510
start=510
end=1020
start=1020
end=1347


In [10]:
probs_list

[tensor([[0.1272, 0.8334, 0.0394]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.3841, 0.5079, 0.1080]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.7310, 0.2039, 0.0652]], grad_fn=<SoftmaxBackward0>)]

Each section has been assign varying levels of sentiment. The first and section sections both score negatively (index 1) and the final sections scores positively (index 0). To calculate the average sentiment across the full text, we will merge these tensors using the stack method:

In [13]:
stacks = torch.stack(probs_list)
stacks

tensor([[[0.1272, 0.8334, 0.0394]],

        [[0.3841, 0.5079, 0.1080]],

        [[0.7310, 0.2039, 0.0652]]], grad_fn=<StackBackward0>)

From here we will calculate the mean score of each column (positive, negative, and neutral sentiment respectively) using mean(dim=0). But before we do that we must reshape our tensor into a 3x3 shape - it is currently a 3x1x3:

In [15]:
shape = stacks.shape
shape

torch.Size([3, 1, 3])

We can reshape our tensor dimensions using the resize_ method, and use dimensions 0 and 2 of our current tensor shape:

In [17]:
stacks.resize_(shape[0], shape[2])

RuntimeError: cannot resize variables that require grad

When we try to resize our tensor, we will receive this RuntimeError telling us that we cannot resize variables that require grad. What this is referring to is the gradient updates of our model tensors during training. PyTorch cannot calculate gradients for tensors that have been reshaped. Fortunately, we don't actually want to use this tensor during any training, so we can use the torch.no_grad() namespace to tell PyTorch that we do not want to calculate any gradients.

In [18]:
with torch.no_grad():
    # we must include our stacks operation in here too
    stacks = torch.stack(probs_list)
    # now resize
    stacks = stacks.resize_(stacks.shape[0], stacks.shape[2])
    # finally, we can calculate the mean value for each sentiment class
    mean = stacks.mean(dim=0)
    
mean

tensor([0.4141, 0.5151, 0.0709])

Our final sentiment prediction shows a reasonable balanced sentiment of both positive and negative classes, with a slightly stronger negative sentiment score overall. We can take the argmax too to specify our winning class.

In [20]:
torch.argmax(mean).item()

1